In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import scipy.stats as stats

### Situation 1: Poisson Distribution Based on Observed Simulation Parameters (Tues, 11/22nd, 6-8 PM)

Parameters for arrival rates, service rates, time block periods, number of servers, and time intervals in minutes:

##### 1. block_start and block_end
Description: The start and end times of the simulation block, representing the period during which the arrivals and services were observed.
Purpose: Helps segment the simulation into manageable intervals for analysis.


##### 2. num_customers
Description: The total number of customers that arrived during the time block.
Purpose: Measures the system's demand within the block. This is determined by a Poisson process based on the arrival rate.

##### 3. avg_wait_time
Description: The average time customers had to wait before their service began.
Purpose: Indicates how long customers typically wait in the queue, reflecting the system's responsiveness.

##### 4. max_wait_time
Description: The longest wait time experienced by any customer during the block.
Purpose: Highlights the worst-case scenario for customer experience in the time block. Useful for identifying periods of potential customer dissatisfaction.

##### 5. server_utilization
Description: The proportion of time that servers were actively engaged in serving customers relative to the total available time.
Purpose: Measures the efficiency of server use. Helps balance resource utilization and customer experience.

##### 6. num_machines
Represents the total number of yogurt machines available for servers to use, shared across all servers. Set to 2 in this simulation to mimic limited resources.


In [7]:
# Set random seed for reproducibility
np.random.seed(42)

# Define simulation parameters based on our sampled observations done in-person at Yogurt Park
arrival_rates = [1/1.5, 1, 1/1.2, 1]  # arrivals per minute (inverse of inter-arrival times)
service_rates = [1/2, 1/2, 1/2, 1/1.5]  # service per minute (inverse of service times)
time_blocks = [(6, 6.5), (6.5, 7), (7, 7.5), (7.5, 8)]  # observation intervals in hours
num_servers = 2
num_machines = 2  # limited machines shared by all servers

# Convert time_blocks to minutes for simplicity
time_intervals = [(start * 60, end * 60) for start, end in time_blocks]


One-Day Simulation based on our Observed Parameters:

In [64]:
import numpy as np
import pandas as pd

def simulate_yogurt_park_day(arrival_rates, service_rates, num_servers, num_machines, time_intervals):
    results = []
    total_num_customers = 0
    total_wait_times = []
    total_server_usage = 0
    total_machine_usage = 0

    for (start, end), arrival_rate, service_rate in zip(time_intervals, arrival_rates, service_rates):
        # Generate arrivals based on Poisson process
        arrivals = []
        current_time = start
        while current_time < end:
            inter_arrival_time = np.random.exponential(1 / arrival_rate)
            current_time += inter_arrival_time
            if current_time < end:
                arrivals.append(current_time)

        num_customers = len(arrivals)
        total_num_customers += num_customers
        service_times = np.random.exponential(1 / service_rate, num_customers)
        
        # Simulate service and calculate wait times
        wait_times = []
        next_available_time = [start] * num_servers
        machine_next_available = [start] * num_machines
        machine_usage = [0] * num_machines  # Track machine usage
        
        for arrival, service_time in zip(arrivals, service_times):
            # Find the next available server
            server = np.argmin(next_available_time)
            
            # Find the next available machine
            machine = np.argmin(machine_next_available)
            
            # Calculate wait times for server and machine
            server_wait = max(0, next_available_time[server] - arrival)
            machine_wait = max(0, machine_next_available[machine] - (arrival + server_wait))
            
            # Calculate total wait time (server + machine wait)
            total_wait_time = server_wait + machine_wait
            
            # Update the departure times for server and machine
            departure_time = arrival + total_wait_time + service_time
            next_available_time[server] = departure_time
            machine_next_available[machine] = departure_time
            machine_usage[machine] += service_time  # Track machine usage
            
            wait_times.append(total_wait_time)
            total_wait_times.extend(wait_times) 
            total_server_usage += np.sum(service_times)
            total_machine_usage += np.sum(machine_usage)
        
        # Calculate metrics
        avg_wait_time = np.mean(wait_times) if wait_times else 0
        max_wait_time = max(wait_times) if wait_times else 0
        server_utilization = np.sum(service_times) / (num_servers * (end - start))
        machine_utilization = np.sum(machine_usage) / (num_machines * (end - start))  # Calculate machine utilization
        
        # Record results
        results.append({
            "block_start": start / 60,
            "block_end": end / 60,
            "num_customers": num_customers,
            "avg_wait_time": avg_wait_time,
            "max_wait_time": max_wait_time,
            "server_utilization": server_utilization,
            "machine_utilization": machine_utilization
        })

    weighted_avg_wait_time = np.mean(total_wait_times) if total_wait_times else 0
    weighted_max_wait_time = max(total_wait_times) if total_wait_times else 0
    total_server_utilization = total_server_usage / (num_servers * (time_intervals[-1][1] - time_intervals[0][0]))
    total_machine_utilization = total_machine_usage / (num_machines * (time_intervals[-1][1] - time_intervals[0][0])) 

    return pd.DataFrame(results)

Simulation of 50 Iterations for Averaged Results:

In [66]:
# Simulate for 50 iterations
simulations = []
for _ in range(50):
    simulation_result = simulate_yogurt_park_day(arrival_rates, service_rates, num_servers, num_machines, time_intervals)
    simulations.append(simulation_result)

# Combine results across iterations
combined_results = pd.concat(simulations, keys=range(1, 51), names=["Simulation", "Index"])

# Compute averages across simulations
average_results = combined_results.groupby(level="Index").mean()


# Display final averaged results
print("Averaged Simulation Results Across 50 Runs:\n")
print(average_results)



Averaged Simulation Results Across 50 Runs:

       block_start  block_end  num_customers  avg_wait_time  max_wait_time  \
Index                                                                        
0              6.0        6.5          20.38       1.063321       3.748829   
1              6.5        7.0          29.14       2.574037       7.075266   
2              7.0        7.5          24.64       1.757417       5.309918   
3              7.5        8.0          30.52       1.094453       3.699458   

       server_utilization  machine_utilization  
Index                                           
0                0.697584             0.697584  
1                0.971002             0.971002  
2                0.816113             0.816113  
3                0.745622             0.745622  


### Situation 2: Opening Up the Inside of Yogurt Park to Accommodate 2 Additional Servers

Congestion Delay (congestion_delay_mean):
Represents the additional delay caused by congestion in the workspace due to limited machines and shared resources. Modeled as an exponential distribution with a mean of 0.2 minutes.

Walking Time (walking_time_mean):
Accounts for the time servers spend walking between workstations or machines. Modeled as an exponential distribution with a mean of 0.1 minutes.

In [5]:
import numpy as np
import pandas as pd

# Set random seed for reproducibility
np.random.seed(42)

# Define simulation parameters for Situation 2
arrival_rates = [1/1.5, 1, 1/1.2, 1]  # arrivals per minute (inverse of inter-arrival times)
service_rates = [1/2, 1/2, 1/2, 1/1.5]  # service per minute (inverse of service times)
time_blocks = [(6, 6.5), (6.5, 7), (7, 7.5), (7.5, 8)]  # observation intervals in hours
num_servers = 4  # total servers increased to 4 (2 inside, 2 outside)
num_machines = 2  # limited machines shared by all servers
congestion_delay_mean = 0.2  # average congestion delay in minutes
walking_time_mean = 0.1  # average walking delay in minutes

# Convert time_blocks to minutes for simplicity
time_intervals = [(start * 60, end * 60) for start, end in time_blocks]

In [6]:
# Function to simulate a single day's operations with congestion
def simulate_yogurt_park_day_with_realistic_factors(arrival_rates, service_rates, num_servers, num_machines, 
                                                    congestion_delay_mean, walking_time_mean, time_intervals):
    results = []

    for (start, end), arrival_rate, service_rate in zip(time_intervals, arrival_rates, service_rates):
        # Generate arrivals based on Poisson process
        arrivals = []
        current_time = start
        while current_time < end:
            inter_arrival_time = np.random.exponential(1 / arrival_rate)
            current_time += inter_arrival_time
            if current_time < end:
                arrivals.append(current_time)

        num_customers = len(arrivals)
        service_times = np.random.exponential(1 / service_rate, num_customers)
        
        # Simulate service and calculate wait times
        wait_times = []
        next_available_time = [start] * num_servers
        machine_next_available = [start] * num_machines
        for arrival, service_time in zip(arrivals, service_times):
            # Find the next available server
            server = np.argmin(next_available_time)
            
            # Find the next available machine
            machine = np.argmin(machine_next_available)
            
            # Calculate congestion delay
            congestion_delay = np.random.exponential(congestion_delay_mean)
            
            # Calculate walking delay
            walking_delay = np.random.exponential(walking_time_mean)
            
            # Calculate total wait time (server, machine, congestion, walking)
            server_wait = max(0, next_available_time[server] - arrival)
            machine_wait = max(0, machine_next_available[machine] - (arrival + server_wait))
            total_wait_time = server_wait + machine_wait + congestion_delay + walking_delay
            
            # Update the departure times for server and machine
            departure_time = arrival + total_wait_time + service_time
            next_available_time[server] = departure_time
            machine_next_available[machine] = departure_time
            
            wait_times.append(total_wait_time)
        
        # Calculate metrics
        avg_wait_time = np.mean(wait_times) if wait_times else 0
        max_wait_time = max(wait_times) if wait_times else 0
        server_utilization = np.sum(service_times) / (num_servers * (end - start))
        machine_utilization = np.sum(service_times) / (num_machines * (end - start))
        
        # Record results
        results.append({
            "block_start": start / 60,
            "block_end": end / 60,
            "num_customers": num_customers,
            "avg_wait_time": avg_wait_time,
            "max_wait_time": max_wait_time,
            "server_utilization": server_utilization,
            "machine_utilization": machine_utilization
        })

    return pd.DataFrame(results)

# Run the enhanced simulation
simulation_results = simulate_yogurt_park_day_with_realistic_factors(
    arrival_rates, service_rates, num_servers, num_machines, congestion_delay_mean, walking_time_mean, time_intervals
)



In [7]:
# Simulate for 50 iterations
simulations = []
for _ in range(50):
    simulation_result = simulate_yogurt_park_day_with_realistic_factors(arrival_rates, service_rates, num_servers, num_machines, 
                                                    congestion_delay_mean, walking_time_mean, time_intervals)
    simulations.append(simulation_result)

# Combine results across iterations
combined_results = pd.concat(simulations, keys=range(1, 51), names=["Simulation", "Index"])

# Compute averages across simulations
average_results = combined_results.groupby(level="Index").mean()

# Display final averaged results
print("Averaged Simulation Results Across 50 Runs:\n")
print(average_results)



Averaged Simulation Results Across 50 Runs:

       block_start  block_end  num_customers  avg_wait_time  max_wait_time  \
Index                                                                        
0              6.0        6.5          19.38       1.597439       4.597288   
1              6.5        7.0          28.70       3.515608       8.948054   
2              7.0        7.5          26.28       3.243975       7.757472   
3              7.5        8.0          30.84       2.470684       6.252610   

       server_utilization  machine_utilization  
Index                                           
0                0.314645             0.629289  
1                0.495685             0.991369  
2                0.449077             0.898155  
3                0.394069             0.788139  


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=a25d82b5-31d5-48ff-8469-cdb2461d6891' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>